In [8]:
from pathlib import Path

import pandas as pd

In [9]:
REPO_ROOT = Path().absolute().parent
MODELS_DIR = REPO_ROOT / "models"
MODELS_DIR

PosixPath('/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/nas/models')

In [10]:
def read_results(dir_: Path, dataset: str | None = None, epochs: str | None = None):
    dataset = dataset or "*"
    epochs = epochs or "*"
    df_list = []
    for path in dir_.glob(f"{dataset}_{epochs}.parquet"):
        path_dataset, path_epochs = path.stem.split("_")
        path_df = pd.read_parquet(path)
        path_df["dataset"] = path_dataset
        path_df["epochs"] = int(path_epochs)
        df_list.append(path_df)
    return pd.concat(df_list).convert_dtypes()

In [11]:
result_df = read_results(MODELS_DIR, dataset="imagenet")
result_df

,index,flops,size_parameters,latency,train.loss,train.accuracy,train.time_per_epoch,train.time,val.loss,val.accuracy,val.time_per_epoch,val.time,test.loss,test.accuracy,test.time_per_epoch,test.time,dataset,epochs
0,1,28.49548,0.809576,0.013774,1.459219,60.33619,65.826919,13165.383867,2.22976,44.488889,0.450504,90.100775,2.266221,44.033333,0.450504,90.100775,imagenet,200
1,2,19.64812,0.566536,0.012761,2.973894,27.219952,68.786171,13757.234145,2.973391,27.633333,0.47691,95.381927,3.015747,27.255556,0.47691,95.381927,imagenet,200
2,3,1.9534,0.080456,0.008155,2.928704,28.455944,25.274706,5054.941255,2.906529,29.433333,0.292387,58.477436,2.972386,28.044444,0.292387,58.477436,imagenet,200
3,4,3.91948,0.136456,0.011799,2.732304,32.191387,39.346727,7869.345344,2.746641,32.144444,0.289579,57.915844,2.78347,31.488889,0.289579,57.915844,imagenet,200
4,5,12.76684,0.379496,0.015982,2.0623,46.20501,60.452025,12090.405048,2.47197,37.877778,0.389245,77.848917,2.49856,38.744444,0.389245,77.848917,imagenet,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15619,15620,1.9534,0.080456,0.009384,3.2214,22.791694,30.57017,366.842043,3.169649,23.133333,0.288019,3.456232,3.218412,23.266667,0.288019,3.456232,imagenet,12
15620,15621,20.63116,0.594536,0.016013,3.167332,23.2294,62.808909,753.706902,3.111229,24.566667,0.418624,5.023489,3.181048,23.433333,0.418624,5.023489,imagenet,12
15621,15622,29.47852,0.837576,0.015286,3.120972,23.156889,70.577927,846.93513,3.073611,24.133333,0.454044,5.448529,3.079313,24.166667,0.454044,5.448529,imagenet,12
15622,15623,1.9534,0.080456,0.010249,3.996838,10.144364,39.843335,478.12002,4.011415,9.7,0.325608,3.907293,4.028404,10.0,0.325608,3.907293,imagenet,12


In [12]:
max_ = {}
for set_ in ("val", "test"):
    max_[set_] = result_df.groupby(["dataset", "epochs"])[f"{set_}.accuracy"].idxmax()
max_

{'val': dataset   epochs
 imagenet  12         2580
           200       10675
 Name: val.accuracy, dtype: int64,
 'test': dataset   epochs
 imagenet  12        2580
           200        856
 Name: test.accuracy, dtype: int64}

In [13]:
result_df.loc[max_["val"]["imagenet"]]

,index,flops,size_parameters,latency,train.loss,train.accuracy,train.time_per_epoch,train.time,val.loss,val.accuracy,val.time_per_epoch,val.time,test.loss,test.accuracy,test.time_per_epoch,test.time,dataset,epochs
2580,2581,38.32588,1.080616,0.022077,1.044194,70.798286,87.308344,17461.668707,2.251974,46.322222,0.546143,109.228552,2.246695,46.477778,0.546143,109.228552,imagenet,200
2580,2581,38.32588,1.080616,0.022077,2.334024,40.043507,87.308344,1047.700122,2.424521,38.666667,0.546143,6.553713,2.446524,38.266667,0.546143,6.553713,imagenet,12
10675,10676,30.46156,0.865576,0.018638,1.183634,67.265326,82.329873,16465.974508,2.193148,46.733333,0.497584,99.516788,2.230778,46.2,0.497584,99.516788,imagenet,200
10675,10676,30.46156,0.865576,0.018638,2.389286,38.735003,82.329873,987.95847,2.46237,36.9,0.497584,5.971007,2.501021,36.166667,0.497584,5.971007,imagenet,12


In [14]:
result_df.loc[max_["test"]["imagenet"]]

,index,flops,size_parameters,latency,train.loss,train.accuracy,train.time_per_epoch,train.time,val.loss,val.accuracy,val.time_per_epoch,val.time,test.loss,test.accuracy,test.time_per_epoch,test.time,dataset,epochs
2580,2581,38.32588,1.080616,0.022077,1.044194,70.798286,87.308344,17461.668707,2.251974,46.322222,0.546143,109.228552,2.246695,46.477778,0.546143,109.228552,imagenet,200
2580,2581,38.32588,1.080616,0.022077,2.334024,40.043507,87.308344,1047.700122,2.424521,38.666667,0.546143,6.553713,2.446524,38.266667,0.546143,6.553713,imagenet,12
856,857,30.46156,0.865576,0.021861,1.223335,66.201274,83.186503,16637.300572,2.166512,46.377778,0.50411,100.822061,2.1815,47.311111,0.50411,100.822061,imagenet,200
856,857,30.46156,0.865576,0.021861,2.418072,38.292683,83.186503,998.238034,2.49952,36.166667,0.50411,6.049324,2.507186,36.433333,0.50411,6.049324,imagenet,12
